In [1]:
# Import system requirements
import sys, os

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time

# DWave Credentials
dwave_solver = os.getenv('DWAVE_API_SOLVER')
api_key = os.getenv('DWAVE_API_TOKEN')

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

from dwave_qbsolv import QBSolv
import neal

import dwave_networkx as dnx

# Use NetworkX graphic package to create a signed network
import networkx as nx

# Select a D-Wave system and handle mapping from problem graph to sampler graph
sampler = EmbeddingComposite(DWaveSampler(solver=dwave_solver, token=api_key))

In [2]:
# Import data

dataset = 'sp100' # values: 'asset_class','ftse','sectors','sp100'

os.chdir("../data_modified")

corr_tensor = np.load('%s_corr.npy' % (dataset)) #list of correlation matrices for each date
dates = np.load('%s_dates.npy' % (dataset), allow_pickle=True) #list of timestamps
nodes = np.load('%s_nodes.npy' % (dataset), allow_pickle=True) #list of tickers

num_examples = corr_tensor.shape[0] #number of dates
dim = corr_tensor.shape[1] #number of assets

#load max clique computation
# max_clique_list = np.load("max_clique_list")
# max_clique_list

In [3]:
# Helper Functions

def make_graph(corr_mat, node_labels, graph_type):

    G = nx.Graph()
    G.add_nodes_from(node_labels)
    dim = corr_mat.shape[0]

    if not dim == len(node_labels):
        raise ValueError('number node labels not = corr matrix dimensions')
        
    if graph_type=='weighted':
        for i in range(dim):
            for j in range(i+1, dim):
                G.add_edge(node_labels[i], node_labels[j], weight=corr_mat[i, j])

    if graph_type=='signed':
        for i in range(dim):
            for j in range(i+1, dim):
                if corr_mat[i,j] < 0:
                    G.add_edge(node_labels[i], node_labels[j], sign=-1)
                elif corr_mat[i,j] > 0:
                    G.add_edge(node_labels[i], node_labels[j], sign=1)
    
    if graph_type=='corr':
        for i in range(dim):
            for j in range(i+1, dim):
                if corr_mat[i,j] != 0.000:
                    G.add_edge(node_labels[i], node_labels[j])
    
    if graph_type=='uncorr':
        for i in range(dim):
            for j in range(i+1, dim):
                if corr_mat[i,j] == 0.000:
                    G.add_edge(node_labels[i], node_labels[j])
    
    density = (2*G.number_of_edges())/(G.number_of_nodes()*(G.number_of_nodes() - 1))
                
    return G, density

def get_max_deg(G):
    degree_sequence = sorted([d for n, d in G.degree()], reverse=True)

    return max(degree_sequence)

def clique_size(clique):
    return len(clique)

In [4]:
corr_mat = corr_tensor[int(num_examples/2), :, :].copy() #take the correlation matrix for a specific date (for visualization)
G, density = make_graph(corr_mat, nodes, 'weighted')

In [5]:
A = nx.adjacency_matrix(G).todense()

In [6]:
node_degree = [t[1] for t in list(G.degree(weight ='weight'))]

In [7]:
sum_node_degree = sum(node_degree)

In [8]:
node_degree_T = np.array(node_degree, ndmin=2)
node_degree = np.transpose(node_degree_T)

In [9]:
B = A - (node_degree*node_degree_T)/sum_node_degree

In [10]:
h = {}
for i in range(len(list(G.nodes()))):
    h[list(G.nodes())[i]] = -B[i,i]

In [11]:
J = {}
counter = 0
for i in range(len(B)):
    for j in range(i+1, len(B)):
        J[list(G.edges())[counter]] = -2*B[i,j]
        counter += 1
        

In [13]:
response = QBSolv().sample_ising(h,J, solver=neal.SimulatedAnnealingSampler()) 

In [14]:
list(response.samples())

[{'AAPL': -1, 'ABBV': 1, 'ABT': -1, 'ACN': -1, 'AGN': 1, 'AIG': -1, 'ALL': -1, 'AMGN': 1, 'AMZN': -1, 'AXP': 1, 'BA': -1, 'BAC': 1, 'BIIB': 1, 'BK': 1, 'BLK': -1, 'BMY': 1, 'C': -1, 'CAT': -1, 'CELG': 1, 'CL': -1, 'CMCSA': -1, 'COF': 1, 'COP': 1, 'COST': -1, 'CSCO': 1, 'CVS': 1, 'CVX': 1, 'DHR': -1, 'DIS': 1, 'DUK': -1, 'EMR': -1, 'EXC': -1, 'F': -1, 'FB': -1, 'FDX': -1, 'GD': -1, 'GE': 1, 'GILD': 1, 'GM': 1, 'GOOG': -1, 'GOOGL': -1, 'GS': 1, 'HAL': -1, 'HD': 1, 'HON': -1, 'IBM': 1, 'INTC': 1, 'JNJ': -1, 'JPM': 1, 'KMI': 1, 'KO': -1, 'LLY': 1, 'LMT': -1, 'LOW': 1, 'MA': -1, 'MCD': -1, 'MDLZ': -1, 'MDT': -1, 'MET': 1, 'MMM': -1, 'MO': -1, 'MRK': -1, 'MS': 1, 'MSFT': -1, 'NEE': -1, 'NKE': 1, 'ORCL': 1, 'OXY': -1, 'PEP': -1, 'PFE': 1, 'PG': -1, 'PM': -1, 'QCOM': -1, 'RTN': -1, 'SBUX': -1, 'SLB': 1, 'SO': -1, 'SPG': 1, 'T': 1, 'TGT': 1, 'TXN': -1, 'UNH': -1, 'UNP': 1, 'UPS': 1, 'USB': 1, 'UTX': -1, 'V': -1, 'VZ': 1, 'WBA': 1, 'WFC': 1},
 {'AAPL': 1, 'ABBV': -1, 'ABT': 1, 'ACN': 1, 'AGN': -

In [ ]:
response_quantum = QBSolv().sample_ising(h,J, solver=sampler, solver_limit=30) #doesn't work with quantum... fix

embedding timed out


ConnectionError: HTTPSConnectionPool(host='cloud.dwavesys.com', port=443): Max retries exceeded with url: /sapi/problems/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000130B74BFDC8>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

Exception ignored in: 'dwave_qbsolv.qbsolv_binding.solver_callback'
Traceback (most recent call last):
  File "python\dwave_qbsolv\qbsolv_binding.pyx", line 65, in dwave_qbsolv.qbsolv_binding.run_qbsolv.dimod_callback
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dimod\core\sampler.py", line 246, in sample_qubo
    return self.sample(bqm, **parameters)
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dwave\system\composites\embedding.py", line 265, in sample
    response = child.sample(bqm_embedded, **parameters)
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dimod\core\sampler.py", line 177, in sample
    sampleset.record.energy += offset
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dimod\sampleset.py", line 690, in record
    self.resolve()
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dimod\sampleset.py", line 1017, in resolve
    samples = self._result_hook(self._future)
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dwave\system\samplers\dwave_sampler.py"

ConnectionError: HTTPSConnectionPool(host='cloud.dwavesys.com', port=443): Max retries exceeded with url: /sapi/problems/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000130B78AE588>: Failed to establish a new connection: [WinError 10051] A socket operation was attempted to an unreachable network'))

Exception ignored in: 'dwave_qbsolv.qbsolv_binding.solver_callback'
Traceback (most recent call last):
  File "python\dwave_qbsolv\qbsolv_binding.pyx", line 65, in dwave_qbsolv.qbsolv_binding.run_qbsolv.dimod_callback
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dimod\core\sampler.py", line 246, in sample_qubo
    return self.sample(bqm, **parameters)
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dwave\system\composites\embedding.py", line 265, in sample
    response = child.sample(bqm_embedded, **parameters)
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dimod\core\sampler.py", line 177, in sample
    sampleset.record.energy += offset
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dimod\sampleset.py", line 690, in record
    self.resolve()
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dimod\sampleset.py", line 1017, in resolve
    samples = self._result_hook(self._future)
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dwave\system\samplers\dwave_sampler.py"

...

ConnectionError: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))

Exception ignored in: 'dwave_qbsolv.qbsolv_binding.solver_callback'
Traceback (most recent call last):
  File "python\dwave_qbsolv\qbsolv_binding.pyx", line 65, in dwave_qbsolv.qbsolv_binding.run_qbsolv.dimod_callback
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dimod\core\sampler.py", line 246, in sample_qubo
    return self.sample(bqm, **parameters)
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dwave\system\composites\embedding.py", line 265, in sample
    response = child.sample(bqm_embedded, **parameters)
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dimod\core\sampler.py", line 177, in sample
    sampleset.record.energy += offset
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dimod\sampleset.py", line 690, in record
    self.resolve()
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dimod\sampleset.py", line 1017, in resolve
    samples = self._result_hook(self._future)
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dwave\system\samplers\dwave_sampler.py"

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

Exception ignored in: 'dwave_qbsolv.qbsolv_binding.solver_callback'
Traceback (most recent call last):
  File "python\dwave_qbsolv\qbsolv_binding.pyx", line 65, in dwave_qbsolv.qbsolv_binding.run_qbsolv.dimod_callback
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dimod\core\sampler.py", line 246, in sample_qubo
    return self.sample(bqm, **parameters)
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dwave\system\composites\embedding.py", line 265, in sample
    response = child.sample(bqm_embedded, **parameters)
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dimod\core\sampler.py", line 177, in sample
    sampleset.record.energy += offset
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dimod\sampleset.py", line 690, in record
    self.resolve()
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dimod\sampleset.py", line 1017, in resolve
    samples = self._result_hook(self._future)
  File "C:\Users\dorot\Anaconda3\lib\site-packages\dwave\system\samplers\dwave_sampler.py"

KeyboardInterrupt: 